# Fake news Detection using Logistic Regression, Random forest classifier, gradient boosting classifier and lstm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import re
# import tensorflow as tf
from sklearn.model_selection import train_test_split
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Embedding
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
fake_df=pd.read_csv('/content/Fake_subset4.csv',encoding='latin1')
true_df=pd.read_csv('/content/True_subset1.csv',encoding='latin1')

In [ ]:
true_df['label']=1
fake_df['label']=0

In [ ]:
news = pd.concat([fake_df,true_df],axis=0)

In [ ]:
news.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [ ]:

news= news.drop(['title','subject','date'],axis=1)

In [ ]:
#reshuffling the data
news = news.sample(frac=1)

In [ ]:
news.reset_index(inplace=True)

In [ ]:
news.drop(['index'],axis=1,inplace = True)

In [ ]:
def wordopt(text):
  # Check if the input is a string
    if isinstance(text, str):
    # Convert to lowercase
     text = text.lower()

    # Remove URLs
     text = re.sub(r'https?://\S+|', '', text)

    # Remove HTML tags
     text = re.sub(r'<.*?>', '', text)

    # Remove punctuation
     text = re.sub(r'[^\w\s]', '', text)

    # Remove digits
     text = re.sub(r'\d', '', text)

    # Remove newline characters
     text = re.sub(r'\n', ' ', text)

     return text

news['text'] = news['text'].apply(wordopt)

In [ ]:
news['text'] = news['text'].apply(wordopt)

In [ ]:
x =news['text']
y =news['label']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)

TF-idf model for making embeddings

In [ ]:
vectorization = TfidfVectorizer()

In [ ]:
# Replace None values with empty strings
x_train_preprocessed = ["" if x is None else x for x in x_train]

# Perform vectorization on preprocessed data
xv_train = vectorization.fit_transform(x_train_preprocessed)

In [ ]:
xv_test = vectorization.transform(x_test)

Logistic Regression Implementation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
LR = LogisticRegression()

In [ ]:
LR.fit(xv_train, y_train)

LogisticRegression()

In [ ]:
pred_lr = LR.predict(xv_test)

In [ ]:
LR.score(xv_test,y_test)

0.9946292446292446

In [ ]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2726
           1       0.99      1.00      0.99      3046

    accuracy                           0.99      5772
   macro avg       0.99      0.99      0.99      5772
weighted avg       0.99      0.99      0.99      5772



In [ ]:
print(len(pred_lr))

5772


In [ ]:
print(len(y_test))

5772


RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(xv_train,y_train)

RandomForestClassifier()

In [ ]:
predict_rfc = rfc.predict(xv_test)

In [ ]:
rfc.score(xv_test, y_test)

0.9980942480942481

In [ ]:
print(classification_report(y_test,predict_rfc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2726
           1       1.00      1.00      1.00      3046

    accuracy                           1.00      5772
   macro avg       1.00      1.00      1.00      5772
weighted avg       1.00      1.00      1.00      5772



Gradient boosting classifier

In [ ]:
gbc = GradientBoostingClassifier()

In [ ]:
gbc.fit(xv_train, y_train)

GradientBoostingClassifier()

In [ ]:
pred_gbc = gbc.predict(xv_test)

In [ ]:
gbc.score(xv_test, y_test)

0.9991337491337491

In [ ]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2726
           1       1.00      1.00      1.00      3046

    accuracy                           1.00      5772
   macro avg       1.00      1.00      1.00      5772
weighted avg       1.00      1.00      1.00      5772



In [ ]:
def output_label(n):
  if n==0:
    return "It is a Fake News."
  elif n==1:
    return "It is a Genuine News."

In [ ]:
def manual_testing(news):
  testing_news ={"text" : [news]}
  news_def_test = pd.DataFrame(testing_news)
  news_def_test["text"] = news_def_test["text"].apply(wordopt)
  news_x_test = news_def_test["text"]
  news_xv_test = vectorization.transform(news_x_test)
  pred_lr = LR.predict(news_xv_test)
  pred_gbc = gbc.predict(news_xv_test)
  pred_rfc = rfc.predict(news_xv_test)

  return "\n\nLR Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(
      output_label(pred_lr[0]) , output_label(pred_gbc[0]), output_label(pred_rfc[0])
  )


In [ ]:
news_article = str(input())

sachin is not in the match


In [ ]:
manual_testing(news_article)

'\n\nLR Prediction: It is a Fake News. \nGBC Prediction: It is a Fake News. \nRFC Prediction: It is a Genuine News.'

# LSTM approach

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Dense

In [ ]:
vocab_size=5000

In [ ]:
messages=x.copy()
messages

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

nltk.download('stopwords')

In [ ]:
text=messages[0].split()
len(text)

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

corpus=[]

max_sent_length=0

for i in range(0, len(messages)):
    print("Sentence no: ",i)

    review=re.sub('[^a-zA-Z]',' ', messages[i])
    review=review.lower()
    review=review.split()

    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

    sentence=messages[i].split()

    if(max_sent_length<len(sentence)):
        max_sent_length=len(sentence)
        print(max_sent_length)

In [ ]:
print(max_sent_length)

In [ ]:
onehot_repr=[one_hot(words,vocab_size)for words in corpus]

In [ ]:
sent_length=max_sent_length
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
len(embedded_docs)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=['accuracy'])

print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
x_final=np.array(embedded_docs)
y_final=np.array(y)

print(x_final.shape)
print(y_final.shape)

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=42)

In [ ]:
history=model.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=10,batch_size=64)

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
testing_sent=news['text'][18000]  ## random sentence for testing

review=re.sub('[^a-zA-Z]',' ', testing_sent)
review=review.lower()
review=review.split()

review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
review=' '.join(review)

testing_onehot_repr=[one_hot(review,vocab_size)]

testing_embedded_docs=pad_sequences(testing_onehot_repr,padding='pre',maxlen=sent_length)

testing_final=np.array(testing_embedded_docs)

print(testing_final.shape)

res=model.predict(testing_final)
print(res)